## 66 - Diophantine Equation
> Consider quadratic Diophantine equations of the form:$$x^2 - Dy^2 = 1$$
    <p>For example, when $D=13$, the minimal solution in $x$ is $649^2 - 13 \times 180^2 = 1$.</p>
    <p>It can be assumed that there are no solutions in positive integers when $D$ is square.</p>
    <p>By finding minimal solutions in $x$ for $D = \{2, 3, 5, 6, 7\}$, we obtain the following:</p>$$\begin{align}
    3^2 - 2 \times 2^2 &= 1\\
    2^2 - 3 \times 1^2 &= 1\\
    {\color{red}{\mathbf 9}}^2 - 5 \times 4^2 &= 1\\
    5^2 - 6 \times 2^2 &= 1\\
    8^2 - 7 \times 3^2 &= 1
    \end{align}$$
    <p>Hence, by considering minimal solutions in $x$ for $D \le 7$, the largest $x$ is obtained when $D=5$.</p>
    <p>Find the value of $D \le 1000$ in minimal solutions of $x$ for which the largest value of $x$ is obtained.</p>

We are looking for [Fundamental solutions of Pell's equation](https://en.wikipedia.org/wiki/Pell%27s_equation#Fundamental_solution_via_continued_fractions). It can be computed with the convergents of the continued fraction for $\sqrt D$. So I reused the code of the two previous problems.

In [1]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return abs(a)

class Fraction:
    def reduce(self):
        d = gcd(self.numerator, self.denominator)
        self.numerator //= d
        self.denominator //= d
    def __init__(self, numerator, denominator):
        self.numerator = numerator
        self.denominator = denominator
        self.reduce()
    def sum(self, x):
        num = self.numerator * x.denominator + self.denominator * x.numerator
        denom = self.denominator * x.denominator
        return Fraction(num, denom)
    def inverse(self):
        return Fraction(self.denominator, self.numerator)

class Zsqrt:
    def __init__(self, params): # Zsqrt((N, a, b, c)) represents (a * sqrt(N) + b)/c
        self.N = params[0]
        self.a = params[1]
        self.b = params[2]
        self.c = params[3]
    def floor(self):
        return int((self.a * self.N**0.5 + self.b) / self.c)
    def next(self):
        k = self.b - self.c * self.floor()
        nexta = self.c * self.a
        nextb = -self.c * k
        nextc = self.a**2 * self.N - k**2
        d = gcd(nexta, gcd(nextb, nextc))
        return Zsqrt((self.N,nexta//d, nextb//d, nextc//d))
    def to_tuple(self):
        return (self.N, self.a, self.b, self.c)

def coeffs(D):
    f = Zsqrt((D, 1, 0, 1))
    c = [f.floor()]
    f = f.next()
    x = f.floor()
    stop = f.to_tuple()
    while True:
        c.append(x)
        f = f.next()
        x = f.floor()
        if f.to_tuple() == stop:
            break
    return c

def convergent(c): # returns the numerator of c[0] + 1 / (c[1] + 1 / (... + 1/c[-1]))
    f = Fraction(0,1)
    for i in range(len(c)-1):
        f = f.sum(Fraction(c[-i-1], 1)).inverse()
    f = f.sum(c[0])
    return f.numerator

next_square = 2
largest_x = 0
D0 = 0

for D in range(2, 1001):
    if D != next_square**2:
        c = coeffs(D)
        if len(c) % 2 == 1:
            c.pop()
        else:
            c = c + c[1:-1]
        x = convergent(c)
        if x > largest_x:
            largest_x = x
            D0 = D
    else:
        next_square += 1

print(D0)

661
